In [3]:
import numpy as np
import pandas as pd

In [5]:
import re
import unicodedata

def remove_english_characters(text):
    """Remove English characters (a-z, A-Z) from text while keeping Urdu and digits"""
    return re.sub(r'[a-zA-Z]', '', text)

def normalize_unicode(text):
    """Normalize Unicode to NFC form for consistent representation"""
    return unicodedata.normalize('NFC', text)

def standardize_punctuation(text):
    """Standardize common punctuation marks"""
    # Replace various dashes and hyphens with a standard one
    text = re.sub(r'[-–—]', '-', text)
    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    # Remove extra punctuation at the end
    text = re.sub(r'([۔؟!])\1+', r'\1', text)
    return text

def preprocess_urdu_text(text):
    """Apply all preprocessing steps"""
    text = remove_english_characters(text)
    text = normalize_unicode(text)
    text = standardize_punctuation(text)
    return text.strip()

# Test the functions with sample data
df = pd.read_csv('all_urdu_moral_stories.csv')
print("Original text sample:")
print(df['content'])
print("\n" + "="*50 + "\n")
print("After preprocessing:")
print(preprocess_urdu_text(df['content']))

Original text sample:
0      ارسلان خان\nشیر جنگل میں آرام کر رہا تھا کہ اس...
1      جاوید اقبال\nایک دن دوستوں کا بیٹھے بیٹھے گاؤں...
2      محمد سراج الدین\nمیں ایک بلی ہوں۔میرا رنگ سفید...
3      محمد فاروق دانش\n”ارے بے وقوف! بات تو سنو․․․․“...
4      فرحی نعیم\nسامنے والے اسلم انکل جب سے ابو جان ...
                             ...                        
104    زبیدہ سلطانہ\nنتھوایک خانہ بدوش قبیلے سے تعلق ...
105    بینش جمیل:\nبہت مدت پہلے دور دراز کے ایک ملک م...
106    جاوید اقبال :\nلمبی اور بھاری چونج والا پرندہ ...
107    میں ایک صاف ستھرے گھر میں رہتا تھا۔ میرا مالک ...
108    جاوید بسام:\nوہ دن میاں بلاقی کیلئے انتہائی حی...
Name: content, Length: 109, dtype: object


After preprocessing:


TypeError: expected string or bytes-like object

In [6]:
# Apply preprocessing to all content and save to new CSV
df = pd.read_csv('all_urdu_moral_stories.csv')

# Apply preprocessing to the content column
df['content_cleaned'] = df['content'].apply(preprocess_urdu_text)

# Save the preprocessed data to a new CSV file
output_file = 'all_urdu_moral_stories_preprocessed.csv'
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Preprocessing complete!")
print(f"Total rows processed: {len(df)}")
print(f"Output saved to: {output_file}")
print("\nSample comparison:")
print(f"Original: {df['content'].iloc[0][:100]}")
print(f"Cleaned:  {df['content_cleaned'].iloc[0][:100]}")

Preprocessing complete!
Total rows processed: 109
Output saved to: all_urdu_moral_stories_preprocessed.csv

Sample comparison:
Original: ارسلان خان
شیر جنگل میں آرام کر رہا تھا کہ اس نے بلی کو دیکھا تو اسے آواز دے کر بلایا۔شیر نے بلی سے 
Cleaned:  ارسلان خان شیر جنگل میں آرام کر رہا تھا کہ اس نے بلی کو دیکھا تو اسے آواز دے کر بلایا۔شیر نے بلی سے 


In [7]:
# Keep only the cleaned content column and save
df_cleaned = df[['content_cleaned']].copy()

# Save to new CSV with only cleaned content
output_file_final = 'all_urdu_moral_stories_cleaned.csv'
df_cleaned.to_csv(output_file_final, index=False, encoding='utf-8')

print(f"Columns removed: title, subtitle, date, content, url")
print(f"Columns kept: content_cleaned")
print(f"Total rows: {len(df_cleaned)}")
print(f"Output saved to: {output_file_final}")
print(f"\nFirst few cleaned samples:")
print(df_cleaned.head())

Columns removed: title, subtitle, date, content, url
Columns kept: content_cleaned
Total rows: 109
Output saved to: all_urdu_moral_stories_cleaned.csv

First few cleaned samples:
                                     content_cleaned
0  ارسلان خان شیر جنگل میں آرام کر رہا تھا کہ اس ...
1  جاوید اقبال ایک دن دوستوں کا بیٹھے بیٹھے گاؤں ...
2  محمد سراج الدین میں ایک بلی ہوں۔میرا رنگ سفید ...
3  محمد فاروق دانش ”ارے بے وقوف! بات تو سنو․․․․“ ...
4  فرحی نعیم سامنے والے اسلم انکل جب سے ابو جان س...


In [8]:
# Define special tokens using unused Unicode bytes
EOS = '\uFFF0'  # End of Sentence
EOP = '\uFFF1'  # End of Paragraph
EOT = '\uFFF2'  # End of Story/Text

print("Special Tokens Defined:")
print(f"EOS (End of Sentence):   U+FFF0 = '{EOS}' (Unicode byte: {ord(EOS)})")
print(f"EOP (End of Paragraph):  U+FFF1 = '{EOP}' (Unicode byte: {ord(EOP)})")
print(f"EOT (End of Story):      U+FFF2 = '{EOT}' (Unicode byte: {ord(EOT)})")

def add_special_tokens(text):
    """Add special tokens to mark sentence, paragraph, and story ends"""
    # Split by Urdu sentence endings and add EOS token
    text = text.replace('۔', f'۔{EOS}')  # Period
    text = text.replace('؟', f'؟{EOS}')  # Question mark
    text = text.replace('!', f'!{EOS}')  # Exclamation
    
    # Split by newlines and add EOP token (paragraphs)
    text = text.replace('\n', f'{EOP}\n')
    
    # Add EOT at the very end
    text = text.rstrip() + EOT
    
    return text

# Load cleaned data and add special tokens
df_cleaned = pd.read_csv('all_urdu_moral_stories_cleaned.csv')

# Apply special tokens
df_cleaned['content_with_tokens'] = df_cleaned['content_cleaned'].apply(add_special_tokens)

# Save with special tokens
output_file_tokens = 'all_urdu_moral_stories_with_tokens.csv'
df_cleaned.to_csv(output_file_tokens, index=False, encoding='utf-8')

print(f"\nTokens added successfully!")
print(f"Output saved to: {output_file_tokens}")
print(f"\nExample with special tokens:")
print(df_cleaned['content_with_tokens'].iloc[0][:300])

Special Tokens Defined:
EOS (End of Sentence):   U+FFF0 = '￰' (Unicode byte: 65520)
EOP (End of Paragraph):  U+FFF1 = '￱' (Unicode byte: 65521)
EOT (End of Story):      U+FFF2 = '￲' (Unicode byte: 65522)

Tokens added successfully!
Output saved to: all_urdu_moral_stories_with_tokens.csv

Example with special tokens:
ارسلان خان شیر جنگل میں آرام کر رہا تھا کہ اس نے بلی کو دیکھا تو اسے آواز دے کر بلایا۔￰شیر نے بلی سے پوچھا ”تم دیکھنے میں تو مجھ جیسی ہو لیکن تمہارا قد اتنا چھوٹا کیوں ہے؟￰“ بلی نے اداسی سے کہا، ”ظالم انسان نے میرا قد چھوٹا کر دیا ہے، ویسے میں رشتے میں تمہاری خالہ ہوں“۔￰ شیر نے یہ سنا تو غصے سے کہا،


In [9]:
# Remove content_cleaned column and keep only content_with_tokens
df_final = pd.read_csv('all_urdu_moral_stories_with_tokens.csv')

# Keep only the content_with_tokens column
df_final = df_final[['content_with_tokens']].copy()

# Rename column for clarity
df_final.rename(columns={'content_with_tokens': 'content'}, inplace=True)

# Save final version
output_file_final = 'all_urdu_moral_stories_with_tokens.csv'
df_final.to_csv(output_file_final, index=False, encoding='utf-8')

print(f"Column removed: content_cleaned")
print(f"Column kept: content_with_tokens (renamed to 'content')")
print(f"Total rows: {len(df_final)}")
print(f"Output saved to: {output_file_final}")
print(f"\nFinal structure:")
print(df_final.head())

Column removed: content_cleaned
Column kept: content_with_tokens (renamed to 'content')
Total rows: 109
Output saved to: all_urdu_moral_stories_with_tokens.csv

Final structure:
                                             content
0  ارسلان خان شیر جنگل میں آرام کر رہا تھا کہ اس ...
1  جاوید اقبال ایک دن دوستوں کا بیٹھے بیٹھے گاؤں ...
2  محمد سراج الدین میں ایک بلی ہوں۔￰میرا رنگ سفید...
3  محمد فاروق دانش ”ارے بے وقوف!￰ بات تو سنو․․․․“...
4  فرحی نعیم سامنے والے اسلم انکل جب سے ابو جان س...
